In the first stage, we generate a batch of high-quality 2D
images using the input text. <br>
For practical implementation, you can either embed the code within the existing stable-dreamfusion model and modify the code in guidance/st_utils.py, or you can write the code separately and use the image and text prompt to train stable-dreamfusion. <br> Here we provide the latter one.

In [ ]:
!pip install --upgrade pip
!pip install --upgrade diffusers transformers accelerate peft

In [ ]:
!mkdir output_image

In [ ]:
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "Lykon/dreamshaper-7"
adapter_id = "latent-consistency/lcm-lora-sdv1-5"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()


prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"

# disable guidance_scale by passing 0
for i in range(5):
  image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0).
  image.save(f"output_image/image_{i}.png")

In the second stage, we use the text and images to create 3D models.

In [ ]:
%cd /content
!unzip stable-dreamfusion_1.zip

/content
Archive:  stable-dreamfusion_1.zip
   creating: stable-dreamfusion_1/
   creating: stable-dreamfusion_1/.git/
  inflating: stable-dreamfusion_1/.git/config  
  inflating: stable-dreamfusion_1/.git/description  
  inflating: stable-dreamfusion_1/.git/HEAD  
   creating: stable-dreamfusion_1/.git/hooks/
  inflating: stable-dreamfusion_1/.git/hooks/applypatch-msg.sample  
  inflating: stable-dreamfusion_1/.git/hooks/commit-msg.sample  
  inflating: stable-dreamfusion_1/.git/hooks/fsmonitor-watchman.sample  
  inflating: stable-dreamfusion_1/.git/hooks/post-update.sample  
  inflating: stable-dreamfusion_1/.git/hooks/pre-applypatch.sample  
  inflating: stable-dreamfusion_1/.git/hooks/pre-commit.sample  
  inflating: stable-dreamfusion_1/.git/hooks/pre-merge-commit.sample  
  inflating: stable-dreamfusion_1/.git/hooks/pre-push.sample  
  inflating: stable-dreamfusion_1/.git/hooks/pre-rebase.sample  
  inflating: stable-dreamfusion_1/.git/hooks/pre-receive.sample  
  inflating: sta

In [ ]:
!unzip stable-dreamfusion.zip

Archive:  stable-dreamfusion.zip
   creating: stable-dreamfusion/
   creating: stable-dreamfusion/.git/
  inflating: stable-dreamfusion/.git/config  
  inflating: stable-dreamfusion/.git/description  
  inflating: stable-dreamfusion/.git/HEAD  
   creating: stable-dreamfusion/.git/hooks/
  inflating: stable-dreamfusion/.git/hooks/applypatch-msg.sample  
  inflating: stable-dreamfusion/.git/hooks/commit-msg.sample  
  inflating: stable-dreamfusion/.git/hooks/fsmonitor-watchman.sample  
  inflating: stable-dreamfusion/.git/hooks/post-update.sample  
  inflating: stable-dreamfusion/.git/hooks/pre-applypatch.sample  
  inflating: stable-dreamfusion/.git/hooks/pre-commit.sample  
  inflating: stable-dreamfusion/.git/hooks/pre-merge-commit.sample  
  inflating: stable-dreamfusion/.git/hooks/pre-push.sample  
  inflating: stable-dreamfusion/.git/hooks/pre-rebase.sample  
  inflating: stable-dreamfusion/.git/hooks/pre-receive.sample  
  inflating: stable-dreamfusion/.git/hooks/prepare-commit-m

In [ ]:
%cd /content/stable-dreamfusion/
!pip install -r requirements.txt

/content/stable-dreamfusion
  Cloning https://github.com/NVlabs/nvdiffrast/ to /tmp/pip-req-build-br2u02wm
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast/ /tmp/pip-req-build-br2u02wm
  Resolved https://github.com/NVlabs/nvdiffrast/ to commit 729261dc64c4241ea36efda84fbf532cc8b425b8
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qhp_d600
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qhp_d600
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
%cd stable-dreamfusion/
!cp -r pretrained ..
%cd ..
!mv pretrained stable-dreamfusion_1/

/content/stable-dreamfusion
/content
mv: cannot move 'pretrained' to 'stable-dreamfusion_1/pretrained': Directory not empty


In [ ]:
!pwd

/content/stable-dreamfusion


In [ ]:
%cd stable-dreamfusion/pretrained

[Errno 2] No such file or directory: 'stable-dreamfusion/pretrained'
/content/stable-dreamfusion


In [ ]:
%cd zero123
!wget https://zero123.cs.columbia.edu/assets/zero123-xl.ckpt

/content/stable-dreamfusion/pretrained/zero123
--2025-04-27 08:38:29--  https://zero123.cs.columbia.edu/assets/zero123-xl.ckpt
Resolving zero123.cs.columbia.edu (zero123.cs.columbia.edu)... 128.59.16.27
Connecting to zero123.cs.columbia.edu (zero123.cs.columbia.edu)|128.59.16.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15465973531 (14G) [application/zip]
Saving to: ‘zero123-xl.ckpt’

zero123-xl.ckpt     100%[===================>]  14.40G  14.6MB/s    in 17m 12s 

2025-04-27 08:55:42 (14.3 MB/s) - ‘zero123-xl.ckpt’ saved [15465973531/15465973531]



In [ ]:
%cd ..
!mkdir omnidata
%cd omnidata

/content/stable-dreamfusion
/content/stable-dreamfusion/omnidata


In [ ]:
%cd /content/stable-dreamfusion/pretrained/omnidata/
!gdown '1Jrh-bRnJEjyMCS7f-WsaFlccfPjJPPHI&confirm=t' # omnidata_dpt_depth_v2.ckpt
!gdown '1wNxVO4vVbDEMEpnAi_jwQObf2MFodcBR&confirm=t' # omnidata_dpt_normal_v2.ckpt

/content/stable-dreamfusion/pretrained/omnidata
Downloading...
From: https://drive.google.com/uc?id=1Jrh-bRnJEjyMCS7f-WsaFlccfPjJPPHI&confirm=t
To: /content/stable-dreamfusion/pretrained/omnidata/omnidata_dpt_depth_v2.ckpt
100% 1.95G/1.95G [00:36<00:00, 53.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wNxVO4vVbDEMEpnAi_jwQObf2MFodcBR&confirm=t
To: /content/stable-dreamfusion/pretrained/omnidata/omnidata_dpt_normal_v2.ckpt
100% 1.95G/1.95G [00:32<00:00, 60.1MB/s]


In [ ]:
%cd /content/stable-dreamfusion/

/content/stable-dreamfusion


In [ ]:
# Import necessary libraries
import os

# Loop 5 times
for i in range(5):
  print(f"Running iteration {i}")
  # Define the main command with the correct image path
  command_1 = f"!python preprocess_image.py output_image/image_{i}.png"
  os.system(command_1)
  command_2 = f"!python main.py -O --image output_image/image_{i}_rgba.png --workspace f_image --iters 5000"
  os.system(command_2)

The result can then be found in the folder named by the image.

The below are the example using one image. If the loop above doesn't work smoothly, you can try to process images one by one.

In [ ]:
!python preprocess_image.py f.png

[INFO] loading image...
[INFO] background removal...
[INFO] depth estimation...
/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(
[INFO] normal estimation...
/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(
[INFO] recenter...


In [ ]:
!python main.py -O --image /content/stable-dreamfusion/f_rgba.png --workspace f_image --iters 5000